In [1]:
#@title Download NTIRE202 validation dataset (toy example)
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1IK2emmr39ribjE7zeIGW221u8na8_ar2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1IK2emmr39ribjE7zeIGW221u8na8_ar2" -O NTIRE2020_Validation_Spectral.zip && rm -rf /tmp/cookies.txt
!mkdir /content/data
!unzip NTIRE2020_Validation_Spectral -d /content/data

--2023-01-12 15:44:59--  https://docs.google.com/uc?export=download&confirm=t&id=1IK2emmr39ribjE7zeIGW221u8na8_ar2
Resolving docs.google.com (docs.google.com)... 74.125.24.139, 74.125.24.101, 74.125.24.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/79fc7gf148b6b32arb4amsfspchjnd0m/1673538225000/14395628383364312969/*/1IK2emmr39ribjE7zeIGW221u8na8_ar2?e=download&uuid=64347988-11fa-41c3-9191-ca4d6b4e9d78 [following]
--2023-01-12 15:44:59--  https://doc-08-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/79fc7gf148b6b32arb4amsfspchjnd0m/1673538225000/14395628383364312969/*/1IK2emmr39ribjE7zeIGW221u8na8_ar2?e=download&uuid=64347988-11fa-41c3-9191-ca4d6b4e9d78
Resolving doc-08-88-docs.googleusercontent.com (doc-08-88-docs.googleusercontent.com)... 74.125.24.132, 2404:

In [2]:
#@title Clone JR2net Github Repository
!git clone https://github.com/bemc22/JR2net
%cd JR2net

Cloning into 'JR2net'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 95 (delta 44), reused 35 (delta 12), pack-reused 0
Unpacking objects: 100% (95/95), done.
/content/JR2net


In [3]:
import os
import tensorflow as tf
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from dataset import get_csi_pipeline, get_val_csi
from jr2net.utils import dd_cassi , dd_icassi, coded2DTO3D

In [4]:
RGB = [27, 17 , 4] # RGB channels
BATCH_SIZE = 3     # Amount of images samples
PATCHES = 9        # The final batch size is BATCH_SIZE * PATCHES
split = 0.9
INPUT_SHAPE = (104, 104, 31)
DATASE_SIZE = (482, 512, 31)

FOLDER_PATH = "/content"

data_path = os.path.join(FOLDER_PATH, 'data')
test_path = data_path

DATA__PARAMS = dict(
    input_size = INPUT_SHAPE,
    batch_size = BATCH_SIZE,
    origin_size = DATASE_SIZE,
)


train_ds = get_csi_pipeline(data_path=data_path, buffer_size=10, **DATA__PARAMS, factor=PATCHES)
# val_ds = get_csi_pipeline(data_path=test_path, buffer_size=None, **DATA__PARAMS)
val_ds = get_val_csi(data_path)

In [5]:
from gc import callbacks
from jr2net.models import JR2net
from jr2net.metrics import prior_loss, psnr
from jr2net.utils import ClearMemory
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau
from tensorflow.keras import backend as k

STAGES = 7
FACTORS = [1, 1, 1/2, 1/2, 1/4, 1/8]
EPOCHS = 10

MODEL_PARAMS = {
    'loss': ['mse', 'mse'],
    'optimizer': tf.keras.optimizers.Adam(learning_rate=1e-4, amsgrad=False),
    'metrics': [psnr]
}


UNROLLED_PARAMS = {
    'input_size' : (None, None, INPUT_SHAPE[-1]),
    'num_stages': STAGES,
    'factors': FACTORS,
    'training': True,
}

unrolled_weights = "unrolled_weights.h5"
callbacks = [ModelCheckpoint(unrolled_weights,
                             monitor='recons_psnr',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
             ClearMemory(),
             ReduceLROnPlateau(monitor="recons_psnr",
                               factor=0.9,
                               patience=100,
                               verbose=0,
                               mode="max")]

main_model = JR2net(**UNROLLED_PARAMS)
model = main_model.unrolled
model.compile(**MODEL_PARAMS)
model.summary()


Model: "JR2net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 spectral_image (InputLayer)    [(None, None, None,  0           []                               
                                 31)]                                                             
                                                                                                  
 coded_aperture (InputLayer)    [(None, None, None,  0           []                               
                                 31)]                                                             
                                                                                                  
 forward_cassi (Lambda)         (None, None, None,   0           ['spectral_image[0][0]',         
                                1)                                'coded_aperture[0][0]',    

In [6]:
model.fit(train_ds, epochs=EPOCHS, validation_data=None, callbacks=None)

Epoch 1/10
3/3 [==============================] - 23s 2s/step - loss: 0.1536 - recons_loss: 0.0768 - repre_loss: 0.0768 - recons_psnr: 13.8206 - repre_psnr: 13.8227
Epoch 2/10
3/3 [==============================] - 6s 2s/step - loss: 0.1495 - recons_loss: 0.0750 - repre_loss: 0.0745 - recons_psnr: 13.9074 - repre_psnr: 13.9362
Epoch 3/10
3/3 [==============================] - 6s 2s/step - loss: 0.1418 - recons_loss: 0.0715 - repre_loss: 0.0703 - recons_psnr: 13.7764 - repre_psnr: 13.8544
Epoch 4/10
3/3 [==============================] - 6s 2s/step - loss: 0.1223 - recons_loss: 0.0623 - repre_loss: 0.0601 - recons_psnr: 13.9160 - repre_psnr: 14.0827
Epoch 5/10
3/3 [==============================] - 6s 2s/step - loss: 0.1302 - recons_loss: 0.0673 - repre_loss: 0.0628 - recons_psnr: 14.6824 - repre_psnr: 14.9917
Epoch 6/10
3/3 [==============================] - 6s 2s/step - loss: 0.1117 - recons_loss: 0.0592 - repre_loss: 0.0525 - recons_psnr: 14.5272 - repre_psnr: 15.0490
Epoch 7/10
3/3 